In [1]:
# Standard imports
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from datetime import datetime
import csv

# Imports for curve fitting
from iminuit import Minuit
from scipy.integrate import quad

# Keras imports
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, Dropout, Flatten, Reshape, Conv2D, Conv2DTranspose, Concatenate, Lambda,BatchNormalization, MaxPooling2D, UpSampling2D
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.losses import binary_crossentropy
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.python.keras.backend import set_image_data_format
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [2]:
def load_dataset(a, b):
    #
    num_snaps = 20000
    tdiff = 9.0
    grid_dataset_list = []
    file_index = 1
    #alphas = np.linspace(0.2, 0.8, 4)
    #betas = np.linspace(0.2, 0.8, 4)
    
    grid_pt_data = pd.read_csv("RealTimeSnaps{}alpha{}beta{}tdiff{}.csv".format(num_snaps, a, b, tdiff), header=None)
    grid_pt_data['y'] = [file_index] * grid_pt_data.shape[0]
    grid_dataset_list.append(grid_pt_data)
    print('Alpha, Beta is {} and given label {}'.format((a, b), file_index))
    file_index += 1
    
    grid_dataset = pd.concat(grid_dataset_list)

    X = grid_dataset.iloc[:, :-1]
    y = grid_dataset.iloc[:, -1]
    
    return X, y, file_index

In [3]:
class VAE:
    #
    def __init__(self, idnum, original_dim=100, intermediate_dim1=75, intermediate_dim2=50, latent_dim=1):
        #
        self.original_dim = original_dim
        self.latent_dim = latent_dim
        self.intermediate_dim1 = intermediate_dim1
        self.intermediate_dim2 = intermediate_dim2
        self.idnum = idnum
        
    def sampling(self, args):
        # Unpack arguments
        z_mean, z_log_var = args
        # Get shape of random noise to sample
        epsilon = K.random_normal(shape=K.shape(z_mean))
        # Return samples from latent space p.d.f.
        return z_mean + K.exp(0.5 * z_log_var) * epsilon
    
    def build_vae(self):
        #
        original_dim = self.original_dim
        latent_dim = self.latent_dim
        intermediate_dim1 = self.intermediate_dim1
        intermediate_dim2 = self.intermediate_dim2
        
        # encoder
        inputs = Input(original_dim, name='input')
        x = Dense(intermediate_dim1, activation='relu')(inputs)
        x = Dense(intermediate_dim2, activation='relu')(x)
        z_mean = Dense(latent_dim, name="z_mean")(x)
        z_log_var = Dense(latent_dim, name="z_log_var")(x)
        z = Lambda(self.sampling, output_shape=(latent_dim, ), name='z')([z_mean, z_log_var])
        self.encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
        
        #decoder
        latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
        x = Dense(intermediate_dim2, activation='relu')(latent_inputs)
        x = Dense(intermediate_dim1, activation='relu')(x)
        outputs = Dense(original_dim, activation='sigmoid')(x)
        self.decoder = Model(latent_inputs, outputs, name='decoder')
        
        i = self.encoder.inputs
        if len(i) == 1:
            i = i[0]
            pass
        z = self.encoder(i)[2]
        o = self.decoder(z)
        self.vae = Model(i, o, name='VAE'+str(self.idnum))
        
    def compile_vae(self):
        # Get the latent p.d.f. mean and log-variance output layers from VAE encoder
        encoder   = self.vae.get_layer('encoder')
        z_log_var = encoder.get_layer('z_log_var').output
        z_mean    = encoder.get_layer('z_mean').output

        # Define reconstruction loss
        def reco_loss(y_true, y_pred):
            # Use binary cross-entropy loss
            reco_loss_value = binary_crossentropy(y_true, y_pred) # Averages over axis=-1
            reco_loss_value = K.sum(reco_loss_value)
            return reco_loss_value

        # Define Kullback-Leibler loss with reference to encoder output layers
        def kl_loss(y_true, y_pred):
            kl_loss_value = 0.5 * (K.square(z_mean) + K.exp(z_log_var) - 1. - z_log_var)
            kl_loss_value = K.sum(kl_loss_value, axis=-1)
            return kl_loss_value

        # Define VAE loss
        def vae_loss(y_true, y_pred):
            return reco_loss(y_true, y_pred) + kl_loss(y_true, y_pred)

        self.vae.compile(optimizer='adam', loss=vae_loss, metrics=[reco_loss, kl_loss])
        return
    
    def get_summaries(self):
        #
        return [self.encoder.summary(),
                self.decoder.summary(),
                self.vae.summary()]
    
    def get_architectures(self):
        #
        return [plot_model(self.encoder, show_shapes=True),
                plot_model(self.decoder, show_shapes=True),
                plot_model(self.vae, show_shapes=True)]

In [4]:
lat_length = 100
#zstart = -4.
#zstop = 4.
#znum = 1000
#zrange = np.linspace(zstart, zstop, znum)
num_pts = 100

In [5]:
def config_taus(rhos):
    #
    taus = np.zeros(lat_length)
    for i in range(lat_length):
        if random.random() <= rhos[i]:
            taus[i] = 1
    return taus

In [6]:
def calc_mfpoftaus(rhos, taus):
    #
    return np.prod(rhos**taus * (1-rhos)**(1-taus))

In [7]:
def calc_complexity1(z_samples, decoder):
    avg_1 = 0
    for i in range(num_pts):
        sample0 = random.choice(z_samples)
        rhos0 = decoder.predict(np.array([sample0]))[0]  
        taus0 = config_taus(rhos0)
        avg_0 = 0
        for i in range(num_pts):
            sample1 = random.choice(z_samples)
            rhos1 = decoder.predict(np.array([sample1]))[0]  
            avg_0 += calc_mfpoftaus(rhos1, taus0)
        avg_0 /= num_pts
        avg_1 += np.log(avg_0)
    avg_1 /= num_pts
    
    avg_2 = 0
    for i in range(num_pts):
        sample2 = random.choice(z_samples)
        rhos2 = decoder.predict(np.array([sample2]))[0]
        avg_0b = 0
        for i in range(num_pts):
            sample_prime = random.choice(z_samples)
            rhos_prime = decoder.predict(np.array([sample_prime]))[0]
            taus_prime = config_taus(rhos_prime)
            avg_0b += np.log(calc_mfpoftaus(rhos2, taus_prime))
        avg_0b /= num_pts
        avg_2 += avg_0b
    avg_2 /= num_pts
    
    return avg_1 - avg_2

In [8]:
def calc_complexity2(z_samples, decoder):
    avg_A = 0
    for i in range(num_pts):
        sample_A = random.choice(z_samples)
        rhos_A = decoder.predict(np.array([sample_A]))[0]
        sum_A = np.sum(rhos_A*np.log(rhos_A) + (1-rhos_A)*np.log(1-rhos_A))
        avg_A += sum_A
    avg_A /= num_pts
    
    avg_rhos = 0
    for i in range(num_pts):
        sample_B = random.choice(z_samples)
        rhos_B = decoder.predict(np.array([sample_B]))[0]
        avg_rhos += rhos_B
    avg_rhos /= num_pts
    sum_B = np.sum(avg_rhos*np.log(avg_rhos) + (1-avg_rhos)*np.log(1-avg_rhos))
    
    return avg_A - sum_B

In [9]:
def one_experiment():

    X_trains = []
    X_tests = []
    y_trains = []
    y_tests = []

    X1, y1, file_index1 = load_dataset(0.2, 0.2)
    X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.5)
    X_trains.append(X1_train)
    X_tests.append(X1_test)
    y_trains.append(y1_train)
    y_trains.append(y1_train)

    X2, y2, file_index2 = load_dataset(0.2, 0.4)
    X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.5)
    X_trains.append(X2_train)
    X_tests.append(X2_test)
    y_trains.append(y2_train)
    y_trains.append(y2_train)

    X3, y3, file_index3 = load_dataset(0.4, 0.2)
    X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.5)
    X_trains.append(X3_train)
    X_tests.append(X3_test)
    y_trains.append(y3_train)
    y_trains.append(y3_train)

    X4, y4, file_index4 = load_dataset(0.2, 0.6)
    X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.5)
    X_trains.append(X4_train)
    X_tests.append(X4_test)
    y_trains.append(y4_train)
    y_trains.append(y4_train)

    X5, y5, file_index5 = load_dataset(0.6, 0.2)
    X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.5)
    X_trains.append(X5_train)
    X_tests.append(X5_test)
    y_trains.append(y5_train)
    y_trains.append(y5_train)

    X6, y6, file_index6 = load_dataset(0.2, 0.8)
    X6_train, X6_test, y6_train, y6_test = train_test_split(X6, y6, test_size=0.5)
    X_trains.append(X6_train)
    X_tests.append(X6_test)
    y_trains.append(y6_train)
    y_trains.append(y6_train)

    X7, y7, file_index7 = load_dataset(0.8, 0.2)
    X7_train, X7_test, y7_train, y7_test = train_test_split(X7, y7, test_size=0.5)
    X_trains.append(X7_train)
    X_tests.append(X7_test)
    y_trains.append(y7_train)
    y_trains.append(y7_train)

    X8, y8, file_index8 = load_dataset(0.4, 0.8)
    X8_train, X8_test, y8_train, y8_test = train_test_split(X8, y8, test_size=0.5)
    X_trains.append(X8_train)
    X_tests.append(X8_test)
    y_trains.append(y8_train)
    y_trains.append(y8_train)

    X9, y9, file_index9 = load_dataset(0.8, 0.4)
    X9_train, X9_test, y9_train, y9_test = train_test_split(X9, y9, test_size=0.5)
    X_trains.append(X9_train)
    X_tests.append(X9_test)
    y_trains.append(y9_train)
    y_trains.append(y9_train)

    X10, y10, file_index10 = load_dataset(0.5, 0.8)
    X10_train, X10_test, y10_train, y10_test = train_test_split(X10, y10, test_size=0.5)
    X_trains.append(X10_train)
    X_tests.append(X10_test)
    y_trains.append(y10_train)
    y_trains.append(y10_train)

    X11, y11, file_index11 = load_dataset(0.8, 0.5)
    X11_train, X11_test, y11_train, y11_test = train_test_split(X11, y11, test_size=0.5)
    X_trains.append(X11_train)
    X_tests.append(X11_test)
    y_trains.append(y11_train)
    y_trains.append(y11_train)

    X12, y12, file_index12 = load_dataset(0.8, 0.8)
    X12_train, X12_test, y12_train, y12_test = train_test_split(X12, y12, test_size=0.5)
    X_trains.append(X12_train)
    X_tests.append(X12_test)
    y_trains.append(y12_train)
    y_trains.append(y12_train)

    VAEs = []
    for i in range(12):
        Vae = VAE(i)
        Vae.build_vae()
        Vae.compile_vae()
        VAEs.append(Vae)

    nb_epochs  = 10
    batch_size = 32

    histories = []
    for i in range(len(VAEs)):
        history = VAEs[i].vae.fit(X_trains[i], X_trains[i],
                                  epochs=nb_epochs,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  validation_split=0.2,
                                  verbose=False)
        histories.append(history)

    zsamples_list = []
    for i in range(len(VAEs)):
        _, _, zsamples = VAEs[i].encoder.predict(X_tests[i])
        zsamples_list.append(zsamples)

    complexities1 = []
    for i in range(len(zsamples_list)):
        complexity1 = calc_complexity1(zsamples_list[i], VAEs[i].decoder)
        complexities1.append(complexity1)
        print('Done complexity1: ',i)

    complexities2 = []
    for i in range(len(zsamples_list)):
        complexity2 = calc_complexity2(zsamples_list[i], VAEs[i].decoder)
        complexities2.append(complexity2)
        print('Done complexity2: ',i)

    print('# run 1')
    print('# P,KL,C1,C2')
    for i in range(len(complexities1)):
        print('{},{},{},{}'.format(i,
                                   histories[i].history['val_kl_loss'][-1],
                                   complexities1[i],
                                   complexities2[i]))

    complexities0 = []
    for i in range(len(histories)):
        complexities0.append(histories[i].history['val_kl_loss'][-1])

    # Highest to lowest complexity
    c0_ranked = (-np.array(complexities0)).argsort()
    c1_ranked = (-np.array(complexities1)).argsort()
    c2_ranked = (-np.array(complexities2)).argsort()

    print('validation_KL: ',c0_ranked)
    print('complexity1: ',c1_ranked)
    print('complexity2: ',c2_ranked)
    
    csvrow = []
    for i in range(len(complexities0)):
        csvrow.append(complexities0[i])
        csvrow.append(complexities1[i])
        csvrow.append(complexities2[i])
    
    return csvrow

In [10]:
num_experiments = 10

In [11]:
# write to csv file
csvtitle = ['# Tour of Phase Diagram Complexities']
csvheader = ['experiment',
             'P1C0', 'P1C1', 'P1C2', 'P2C0', 'P2C1', 'P2C2', 'P3C0', 'P3C1', 'P3C2',
             'P4C0', 'P4C1', 'P4C2', 'P5C0', 'P5C1', 'P5C2', 'P6C0', 'P6C1', 'P6C2',
             'P7C0', 'P7C1', 'P7C2', 'P8C0', 'P8C1', 'P8C2', 'P9C0', 'P9C1', 'P9C2',
             'P10C0', 'P10C1', 'P10C2', 'P11C0', 'P11C1', 'P11C2', 'P12C0', 'P12C1', 'P12C2'
             ]

with open('Phase_diagram_complexity_data.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(csvtitle)
    writer.writerow(csvheader)
    for i in range(num_experiments):
        writer.writerow([i+1]+ one_experiment())

Alpha, Beta is (0.2, 0.2) and given label 1
Alpha, Beta is (0.2, 0.4) and given label 1
Alpha, Beta is (0.4, 0.2) and given label 1
Alpha, Beta is (0.2, 0.6) and given label 1
Alpha, Beta is (0.6, 0.2) and given label 1
Alpha, Beta is (0.2, 0.8) and given label 1
Alpha, Beta is (0.8, 0.2) and given label 1
Alpha, Beta is (0.4, 0.8) and given label 1
Alpha, Beta is (0.8, 0.4) and given label 1
Alpha, Beta is (0.5, 0.8) and given label 1
Alpha, Beta is (0.8, 0.5) and given label 1
Alpha, Beta is (0.8, 0.8) and given label 1


C:\Users\Matthew\Anaconda3\envs\daml2\lib\site-packages\tensorflow\python\keras\engine\training.py:2464: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Done complexity1:  0
Done complexity1:  1
Done complexity1:  2
Done complexity1:  3
Done complexity1:  4
Done complexity1:  5
Done complexity1:  6
Done complexity1:  7
Done complexity1:  8
Done complexity1:  9
Done complexity1:  10
Done complexity1:  11
Done complexity2:  0
Done complexity2:  1
Done complexity2:  2
Done complexity2:  3
Done complexity2:  4
Done complexity2:  5
Done complexity2:  6
Done complexity2:  7
Done complexity2:  8
Done complexity2:  9
Done complexity2:  10
Done complexity2:  11
# run 1
# P,KL,C1,C2
0,1.1688141822814941,19.12247637004394,8.826105804443358
1,0.0003275039198342711,0.05652323169368145,-0.02235778808594091
2,0.00018717387865763158,0.44478003946577616,0.003917846679684089
3,0.00023174572561401874,0.2616623712571524,-0.006092567443850783
4,0.00018004875164479017,-0.013142162610719765,0.019018783569336506
5,0.00022949153208173811,0.9581063651401251,-0.008995704650878622
6,7.126660057110712e-05,-1.076544303936899,-0.0002767181396450269
7,5.6611410400364

Done complexity1:  10
Done complexity1:  11
Done complexity2:  0
Done complexity2:  1
Done complexity2:  2
Done complexity2:  3
Done complexity2:  4
Done complexity2:  5
Done complexity2:  6
Done complexity2:  7
Done complexity2:  8
Done complexity2:  9
Done complexity2:  10
Done complexity2:  11
# run 1
# P,KL,C1,C2
0,1.1489439010620117,17.564276688052956,8.793351593017576
1,0.00032126024598255754,0.8303162100683892,-0.020060997009274217
2,6.79790391586721e-05,0.06294104252829413,0.009075431823731606
3,0.00037236593198031187,0.2670277204138074,0.04048954010009709
4,0.00010322063462808728,-0.7396327259970263,-0.05549270629882841
5,0.0002793006715364754,0.1611571053830474,0.029574356079102415
6,0.00010530529107199982,-0.745116207987337,-0.0035445785522441042
7,8.759870252106339e-05,0.46190319502392185,0.010124740600588211
8,6.704065162921324e-05,-0.17341305951909192,-0.002096862792967613
9,6.322535773506388e-05,-0.13238121713351347,0.002712249755859375
10,7.479379564756528e-05,-0.089043

Done complexity2:  8
Done complexity2:  9
Done complexity2:  10
Done complexity2:  11
# run 1
# P,KL,C1,C2
0,1.106223464012146,17.92317200139081,9.219489402770996
1,0.00022187191643752158,1.4543856362703806,-0.09134429931640398
2,0.00013805438356939703,1.3959453563434678,0.01842559814453182
3,0.00029772528796456754,0.7565745273654514,0.014027786254885655
4,6.928388029336929e-05,-0.4462775574855655,0.038457946777342045
5,0.0003101112670265138,0.35757288275077315,-0.028382148742679192
6,0.00011555499804671854,-0.3426626987889989,-0.032754287719725994
7,7.713278318988159e-05,0.16449592001649194,-0.0052246856689492915
8,8.829838043311611e-05,-0.0017291301817863314,-0.0008639526367204553
9,7.91479687904939e-05,0.12216338822243245,0.0033991241455026966
10,0.00018390512559562922,0.0406097273831989,0.003049697875979973
11,3.3892891224240884e-05,-0.1597712692865656,0.0035886383056578097
validation_KL:  [ 0  5  3  1 10  2  6  8  9  7  4 11]
complexity1:  [ 0  1  2  3  5  7  9 10  8 11  6  4]
com